In [4]:
### to deal with missing data 
```{mice package}
libarary(mice)
md.pattern(data)
```




ERROR: Error in parse(text = x, srcfile = src): 不能用零長度的變數名


In [5]:
### to visiualize the missing data 
```{}
library(VIM)
aggr_plot <- aggr(data, col = c('navyblue', 'red'), numbers=TRUE, sortVars=TRUE, 
                  labels=names(data), cex.axis=.7, gap=3, 
                  ylab=c("Histogram of missing data", "Pattern"))
```


ERROR: Error in parse(text = x, srcfile = src): 不能用零長度的變數名


In [8]:
### 参数注解： 1. m=5指的是插补数据集的数量，5是默认值 2. meth='pmm'指的是插补方法。在这里，我们使用预测均值匹配（Predictive mean matching ）作为插补方法。其他插补方法可以通过methods(mice)来查看。
tempData <- mice(data,m=5,maxit=50,meth='pmm',seed=500)
summary(tempData)
### 如果你想查看插补的数据，例如Qzone变量，输入下面的代码行即可。
tempData$imp$Ozone
### 如果需要查看每个变量所用的插补方法，mice可以很容易的做到
tempData$meth
### 现在，我们可以使用complete()函数返回完整的数据集。
completedData <- complete(tempData,1)

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

ERROR: Error in mice(data, m = 5, maxit = 50, meth = "pmm", seed = 500): 沒有這個函數 "mice"


In [9]:
### check the age distribution

par(mfrow=c(1,2))
hist(full$Age, freq=F, main='Age: Original Data', 
  col='darkgreen', ylim=c(0,0.04))
hist(mice_output$Age, freq=F, main='Age: MICE Output', 
  col='lightgreen', ylim=c(0,0.04))

ERROR: Error in hist(full$Age, freq = F, main = "Age: Original Data", col = "darkgreen", : 找不到物件 'full'


In [10]:
### 4.2 Building the model
We then build our model using randomForest on the training set.

# Set a random seed
set.seed(754)

# Build the model (note: not all possible variables are used)
rf_model <- randomForest(factor(Survived) ~ Pclass + Sex + Age + SibSp + Parch + 
                                            Fare + Embarked + Title + 
                                            FsizeD + Child + Mother,
                                            data = train)

# Show model error
plot(rf_model, ylim=c(0,0.36))
legend('topright', colnames(rf_model$err.rate), col=1:3, fill=1:3)

ERROR: Error in parse(text = x, srcfile = src): <text>:2:4: unexpected symbol
1: ### 4.2 Building the model
2: We then
      ^


In [ ]:
### 4.3 Variable importance
Let’s look at relative variable importance by plotting the mean decrease in Gini calculated across all trees.

# Get importance
importance    <- importance(rf_model)
varImportance <- data.frame(Variables = row.names(importance), 
                            Importance = round(importance[ ,'MeanDecreaseGini'],2))

# Create a rank variable based on importance
rankImportance <- varImportance %>%
  mutate(Rank = paste0('#',dense_rank(desc(Importance))))

# Use ggplot2 to visualize the relative importance of variables
ggplot(rankImportance, aes(x = reorder(Variables, Importance), 
    y = Importance, fill = Importance)) +
  geom_bar(stat='identity') + 
  geom_text(aes(x = Variables, y = 0.5, label = Rank),
    hjust=0, vjust=0.55, size = 4, colour = 'red') +
  labs(x = 'Variables') +
  coord_flip() + 
  theme_few()